In [10]:
from dataclasses import dataclass , field
from typing import Literal
@dataclass
class EndStatus:
    name  : str
    epoch : int # epoch of trigger
    
@dataclass
class EndofLoop:
    max_epoch : int = 200
    status : list[EndStatus] = field(default_factory=list)

    def __post_init__(self) -> None: pass
    def __bool__(self): return len(self.status) > 0
    def new_loop(self): self.status = []
    def loop_end(self , epoch):
        if epoch >= self.max_epoch: self.add_status('Max Epoch' , epoch)
    def add_status(self , status : str , epoch : int): 
        self.status.append(EndStatus(status , epoch))
    def trigger_epoch(self):
        return min([sta.epoch for sta in self.status]) if self else self.max_epoch
    
@dataclass
class TrainerStatus:
    epoch   : int = -1
    attempt : int = -1
    round   : int = -1
    model_num  : int = -1
    model_date : int = -1
    model_type : Literal['best' , 'swabest' , 'swalast'] = 'best'
    eol : EndofLoop = field(default_factory=EndofLoop)

    @property
    def case(self):
        return self.epoch > 0


In [11]:
a = TrainerStatus()

In [1]:
import inspect  
  
def example_function(a, b, c=3, *args, **kwargs): 
    print(inspect.signature(example_function)  ) 
    pass  
  
# 获取函数的签名  
sig = inspect.signature(example_function)  
  
# 遍历签名中的所有参数  
for name, param in sig.parameters.items():  
    print(f"Parameter: {name}")  
    if param.default != param.empty:  
        print(f"Default value: {param.default}")  
    if param.kind == inspect.Parameter.POSITIONAL_ONLY:  
        print("This is a positional-only parameter.")  
    elif param.kind == inspect.Parameter.POSITIONAL_OR_KEYWORD:  
        print("This parameter can be positional or keyword.")  
    elif param.kind == inspect.Parameter.VAR_POSITIONAL:  
        print("This is a variable-positional parameter (*args).")  
    elif param.kind == inspect.Parameter.KEYWORD_ONLY:  
        print("This parameter is keyword-only.")  
    elif param.kind == inspect.Parameter.VAR_KEYWORD:  
        print("This is a variable-keyword parameter (**kwargs).")  
  
# 输出函数的全部参数信息  
print("Function parameters:")  
for param in sig.parameters:  
    print(f"{param}: {sig.parameters[param]}")

Parameter: a
This parameter can be positional or keyword.
Parameter: b
This parameter can be positional or keyword.
Parameter: c
Default value: 3
This parameter can be positional or keyword.
Parameter: args
This is a variable-positional parameter (*args).
Parameter: kwargs
This is a variable-keyword parameter (**kwargs).
Function parameters:
a: a
b: b
c: c=3
args: *args
kwargs: **kwargs


In [31]:
from inspect import currentframe
class BaseCB:
    def __init__(self , a) -> None:
        self.a = a
        print(self._infomation())
        
    def _infomation(self):
        args = {k:v for k,v in getattr(currentframe() , 'f_back').f_locals.items() if k not in ['self' , 'model_module'] and not k.startswith('_')}
        return f'Callback of {self.__class__.__name__}' + '({})'.format(','.join([f'{k}={v}' for k,v in args.items()]))

class BasicCallBack(BaseCB):
    def __init__(self , a , b) -> None:
        #args = {k:v for k,v in getattr(inspect.currentframe() , 'f_locals').items() if k != 'self' and not k.startswith('_')}
        #print(self._infomation())# + '({})'.format(','.join([f'{k}={v}' for k,v in args.items()])))
        super().__init__(a)
        self.b = b
        
b = BasicCallBack(1,2)

Callback of BasicCallBack(a=1)


In [3]:
def f():
    pass
if f:
    print(1)

1


In [18]:
import inspect  
  
def example_function(a, b, c=3, *args, **kwargs): 
    print({k:v for k,v in getattr(inspect.currentframe() , 'f_locals').items()})
    pass  

example_function(1,2)

{'a': 1, 'b': 2, 'c': 3, 'args': (), 'kwargs': {}}


In [13]:
c = a.case
a.epoch = 10
c , a.case

(False, True)

In [9]:
a.b

False

In [1]:
import torch
torch.cuda.is_available()

True

In [3]:
import inspect

def f():
    a = A()
    a.__enter__()
    with a:
        print(0)

class A:
    def __enter__(self):
        print(inspect.currentframe().f_back.f_code.co_name)
        print(1)

    def __exit__(self , *args):
        print(2)

f()

class B(A):
    def __init__(self) -> None:
        super().__init__()
        self.b = 10

f
1
f
1
0
2


In [4]:
b = B()
isinstance(b , A)

True

In [1]:
# %% test a specific model
from src.module import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

Use device name: NVIDIA GeForce RTX 4090


24-04-26 00:41:29|MOD:pipeline    |: Model Specifics:
24-04-26 00:41:29|MOD:pipeline    |: Start Process [Data] at Fri Apr 26 00:41:29 2024!


--Process Queue : Data + Fit + Test
--Start Training New!
--Model_name is set to gru_day_ShortTest.31!
Apply Callback of DynamicDataLink() , this is essential for TRA models!
Apply Callback of LoaderDisplay(1) , this is a little bit slow!
Apply Callback of CudaEmptyCache(20) , this is very slow!
Apply Callback of ProcessTimer()
{'random_seed': None,
 'model_name': 'gru_day_ShortTest.31',
 'model_module': 'gru',
 'model_data_type': 'day',
 'model_data_prenorm': {'day': {'divlast': True, 'histnorm': True}},
 'labels': ['std_lag1_10'],
 'beg_date': 20170103,
 'end_date': 20171201,
 'interval': 120,
 'input_step_day': 5,
 'sample_method': 'train_shuffle',
 'shuffle_option': 'epoch'}
{'hidden_dim': [64],
 'seqlens': [{'day': 30, '30m': 30, 'dms': 30}],
 'tra_seqlens': [{'hist_loss': 40}],
 'dropout': [0.1],
 'enc_in': [True],
 'enc_att': [False],
 'rnn_type': ['lstm'],
 'rnn_att': [False],
 'rnn_layers': [2],
 'dec_mlp_layers': [2],
 'num_output': [1],
 'kernel_size': [3],
 'hidden_as_facto

24-04-26 00:41:32|MOD:pipeline    |: Finish Process [Data], Cost 3.3 Secs
24-04-26 00:41:32|MOD:pipeline    |: Start Process [Fit] at Fri Apr 26 00:41:32 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


Train Ep#  0 loss : 1.00674:   5%|▌         | 1/19 [00:00<00:04,  4.27it/s]

score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


Valid Ep#  0 score : -0.05547: 100%|██████████| 20/20 [00:00<00:00, 237.10it/s]
FirstBite Ep#  0 : loss  1.00190, train-0.00794, valid-0.05547, best-0.0555, lr1.0e-07
Valid Ep#  1 score : 0.01339: 100%|██████████| 20/20 [00:00<00:00, 227.50it/s]
FirstBite Ep#  1 : loss  0.97819, train 0.04036, valid 0.01339, best 0.0134, lr1.3e-03
Valid Ep#  2 score : 0.02845: 100%|██████████| 20/20 [00:00<00:00, 245.77it/s]
FirstBite Ep#  2 : loss  0.95214, train 0.05990, valid 0.02845, best 0.0285, lr2.5e-03
Valid Ep#  3 score : 0.06451: 100%|██████████| 20/20 [00:00<00:00, 250.05it/s]
FirstBite Ep#  3 : loss  0.91812, train 0.07970, valid 0.06451, best 0.0645, lr3.8e-03
Valid Ep#  4 score : 0.11137: 100%|██████████| 20/20 [00:00<00:00, 239.50it/s]
FirstBite Ep#  4 : loss  0.88015, train 0.11373, valid 0.11137, best 0.1114, lr5.0e-03
Valid Ep#  5 score : 0.12686: 100%|██████████| 20/20 [00:00<00:00, 243.53it/s]
FirstBite Ep#  5 : loss  0.86209, train 0.13920, valid 0.12686, best 0.1269, lr3.8e-03
Val

                         keys  num_calls  total_time  avg_time
9           on_after_backward       1560        1560       1.0
15    on_validation_batch_end        800         800       1.0
13  on_validation_batch_start        800         800       1.0
14        on_validation_batch        800         800       1.0
7        on_train_batch_start        780         780       1.0
8          on_before_backward        780         780       1.0
10         on_train_batch_end        780         780       1.0
24              on_test_batch        669         669       1.0
25          on_test_batch_end        669         669       1.0
23        on_test_batch_start        669         669       1.0
17         on_after_fit_epoch         40          40       1.0
16    on_validation_epoch_end         40          40       1.0
11         on_train_epoch_end         40          40       1.0
12  on_validation_epoch_start         40          40       1.0
6        on_train_epoch_start         40          40   

In [ ]:
# prepare data
from src.module import DataModule
DataModule.prepare_data()

Use device name: NVIDIA GeForce RTX 4090


24-03-31 09:52:11|MOD:data_preprocessing|: if_train is False , Data Processing start!
24-03-31 09:52:11|MOD:data_preprocessing|: 3 datas :['y', 'trade_day', 'trade_30m']


Sun Mar 31 09:52:11 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.39 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.33 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 1.66 secs
models blocks merging ...... cost 0.17 secs
y blocks process ...... cost 0.88 secs
y blocks masking ...... cost 0.05 secs
y blocks saving  ...... cost 0.14 secs
y blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:14 2024 : y finished! Cost 3.67 Seconds
Sun Mar 31 09:52:14 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 0.56 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.03 secs
trade_day blocks masking ...... cost 0.05 secs
trade_day blocks saving  ...... cost 0.16 secs
trade_day blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:15 2024 : trade_day finished! Cost 0.81 Seconds
Sun Mar 31 09:52:15 2024 : trade_30m start ...
trade blocks r

24-03-31 09:52:21|MOD:data_preprocessing|: Data Processing Finished! Cost 10.02 Seconds


... cost 1.13 secs
trade_30m blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:21 2024 : trade_30m finished! Cost 5.19 Seconds


In [2]:
import inspect

def my_function(a, b):
    pass

# Assert that the function has exactly two arguments with names 'a' and 'b'
assert len(inspect.signature(my_function).parameters) == 2
assert inspect.signature(my_function).parameters['a'].name == 'a'
assert inspect.signature(my_function).parameters['b'].name == 'b'

print("Function 'my_function' has two arguments 'a' and 'b'")


Function 'my_function' has two arguments 'a' and 'b'


In [18]:
[v.name for v in inspect.signature(my_function).parameters.values()]

['a', 'b']